In [39]:
import json

### zip code for 50 states

In [57]:
# having all zip code for 50 states in US
# from ninja api
import requests

states = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA',
        'KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM',
        'NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV',
        'WI','WY']
city = ""
states_zip = {}
for state in states:
    api_url = 'https://api.api-ninjas.com/v1/zipcode?city={}&state={}'.format(city, state)

    response = requests.get(api_url+city, headers={'X-Api-Key': '0+WwzqaBAr5kWjywXArIpA==DlpdDtBfCV9Pb7Nw'})
    if response.status_code == requests.codes.ok:
        states_zip[state] = response.json()
    else:
        print("Error:", response.status_code, response.text)

In [55]:
list(states_zip.keys())[0]

'AL'

In [61]:
import pandas as pd
for key in states_zip.keys():
    df_state = pd.DataFrame.from_dict(states_zip[key])
    states_zip[key] = df_state

In [67]:
df_states = pd.concat(states_zip.values(),ignore_index = True)[['zip_code','city','state','county','timezone','area_codes','lat','lon']]
df_states.to_csv('df_states_zip.csv')

### sales tax

In [1]:
import psycopg2

connection_params = {
    "host":"13.43.84.151",
    "user": "ping-fan.chen.23@ucl.ac.uk",
    "password": "khsLcR",
    "dbname": "postgres",
}

conn = psycopg2.connect(**connection_params)

cur = conn.cursor()

In [30]:
cur = conn.cursor()
cur.execute("SELECT zip_code FROM schema_pingfanchen23uclacuk.zip_code_by_city;")
df = cur.fetchall()
zip_list = [i[0] for i in df]
print(zip_list[:10])
cur.close()

['35004', '35005', '35006', '35007', '35010', '35011', '35013', '35014', '35015', '35016']


In [20]:
import os
current_dir = os.getcwd() # assigns the current working directory to the variable current_dir
print(current_dir)

jar_location = current_dir + "/postgresql-42.3.2.jar"
# Constructs the file path for the PostgreSQL JDBC driver JAR file by appending the filename to the current directory path
jar_location

from pyspark.sql import SparkSession
# Initialize Spark Session
spark = SparkSession.builder \
    .config("spark.driver.extraClassPath", jar_location)\
    .appName("Enhanced Data Warehouse ETL") \
    .getOrCreate()

/home/jovyan/dataengineer


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/05 23:05:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/05 23:05:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [31]:
import requests
df_list = []
for zip_code in zip_list[::100]:
    api_url = 'https://api.api-ninjas.com/v1/salestax?zip_code={}'.format(zip_code)
    response = requests.get(api_url, headers={'X-Api-Key': '0+WwzqaBAr5kWjywXArIpA==DlpdDtBfCV9Pb7Nw'})
    if response.status_code == requests.codes.ok:
        data = response.json()
        df = spark.createDataFrame(data)
        df_list.append(df)
else:
    print("Error:", response.status_code, response.text)

from functools import reduce
from pyspark.sql import DataFrame
df_all = reduce(DataFrame.unionAll, df_list)
# Show the dataframe
df_all.show()

Error: 200 [{"zip_code": "83111", "total_rate": "0.050000", "state_rate": "0.040000", "city_rate": "0.000000", "county_rate": "0.010000", "additional_rate": "0.000000"}]


24/03/05 23:35:55 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
24/03/05 23:35:57 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
24/03/05 23:36:00 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB


+---------------+---------+-----------+----------+----------+--------+
|additional_rate|city_rate|county_rate|state_rate|total_rate|zip_code|
+---------------+---------+-----------+----------+----------+--------+
|       0.020000| 0.040000|   0.000000|  0.040000|  0.100000|   35004|
|       0.020000| 0.040000|   0.000000|  0.040000|  0.100000|   35173|
|       0.000000| 0.030000|   0.030000|  0.040000|  0.100000|   35444|
|       0.000000| 0.045000|   0.010000|  0.040000|  0.095000|   35634|
|       0.000000| 0.040000|   0.010000|  0.040000|  0.090000|   35962|
|       0.000000| 0.035000|   0.025000|  0.040000|  0.100000|   36112|
|       0.000000| 0.035000|   0.025000|  0.040000|  0.100000|   36420|
|       0.000000| 0.000000|   0.010000|  0.040000|  0.050000|   36583|
|       0.000000| 0.040000|   0.010000|  0.040000|  0.090000|   36849|
|       0.000000| 0.040000|   0.000000|  0.000000|  0.040000|   99585|
|       0.000000| 0.000000|   0.000000|  0.000000|  0.000000|   99704|
|     

In [33]:
host="13.43.84.151"
dbname="postgres"
user="ping-fan.chen.23@ucl.ac.uk"
password="khsLcR"
port="5432"
schema="schema_pingfanchen23uclacuk"
from sqlalchemy import create_engine
# JDBC URL
url = f"jdbc:postgresql://{host}:{port}/{dbname}?currentSchema=schema_pingfanchen23uclacuk"

# Connection properties
properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver"
}

# Current directory and JDBC driver location
current_dir = os.getcwd()
jar_location = os.path.join(current_dir, "postgresql-42.3.2.jar")

properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver"
}

df_all.write.jdbc(url=url, table="sale_tax_by_zip", mode="overwrite", properties=properties)

print("Data has been successfully stored in the database")

24/03/05 23:37:40 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB


Data has been successfully stored in the database


In [34]:
# Query to get all table names in a specific schema
cur = conn.cursor()
schema_to_check = 'schema_pingfanchen23uclacuk'  # replace with your actual schema name
cur.execute(f"SELECT table_name FROM information_schema.tables WHERE table_schema = '{schema_to_check}';")
tables = cur.fetchall()

# Print the tables in the schema
print(f"Tables in schema '{schema_to_check}':")
for table in tables:
    print(table[0])

# Close the cursor and the connection
cur.close()
conn.close()

Tables in schema 'schema_pingfanchen23uclacuk':
zip_code_by_city
sale_tax_by_zip


### median price of rental house

In [37]:
median_price = spark.read.option("header", "true").option("startingValue", "2").csv('Metro_median_sale_price_uc_sfrcondo_month.csv')
median_price.show(5)

+--------+--------+---------------+----------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+---------

In [38]:
host="13.43.84.151"
dbname="postgres"
user="ping-fan.chen.23@ucl.ac.uk"
password="khsLcR"
port="5432"
schema="schema_pingfanchen23uclacuk"
from sqlalchemy import create_engine
# JDBC URL
url = f"jdbc:postgresql://{host}:{port}/{dbname}?currentSchema=schema_pingfanchen23uclacuk"

# Connection properties
properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver"
}

# Current directory and JDBC driver location
current_dir = os.getcwd()
jar_location = os.path.join(current_dir, "postgresql-42.3.2.jar")

properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver"
}

median_price.write.jdbc(url=url, table="median_price", mode="overwrite", properties=properties)

print("Data has been successfully stored in the database")

Data has been successfully stored in the database


In [40]:
# Query to get all table names in a specific schema
conn = psycopg2.connect(**connection_params)
cur = conn.cursor()
schema_to_check = 'schema_pingfanchen23uclacuk'  # replace with your actual schema name
cur.execute(f"SELECT table_name FROM information_schema.tables WHERE table_schema = '{schema_to_check}';")
tables = cur.fetchall()

# Print the tables in the schema
print(f"Tables in schema '{schema_to_check}':")
for table in tables:
    print(table[0])

# Close the cursor and the connection
cur.close()
conn.close()

Tables in schema 'schema_pingfanchen23uclacuk':
zip_code_by_city
sale_tax_by_zip
median_price


# IGNORE

In [8]:
# https://www.reddit.com/r/madeinpython/comments/14i106b/i_used_to_use_the_zillow_api_in_my_personal/
# for free acount, only can request for 30 times, maybe we need two account to use this api and save it into json file
import requests

url = "https://zillow56.p.rapidapi.com/market_data"

# city is based on this: https://zh.wikipedia.org/wiki/%E7%BE%8E%E5%9B%BD%E5%90%84%E5%B7%'9E%E6%9C%80%E5%A4%A7%E5%9F%8E%E5%B8%82%E5%88%97%E8%A1%A8
city_list={'al':'birmingham','ak':'anchorage','az':'phoenix','ar':'little rock','ca':'los angeles','co':'denver',
           'ct':'bridgeport','de':'wilmington','fl':'jacksonville','ga':'atlanta','hi':'honolulu',
           'id':'boise','il':'chicago','in':'indianapolis','ia':'des moines','ks':'wichita','ky':'louisville',
           'la':'new orleans','me':'portland','md':'baltimore','ma':'boston','mi':'detroit','mn':'minneapolis','ms':'jackson',
           'mo':'kansas city','mt':'billings','ne':'omaha','nv':'las vegas','nh':'manchester','nj':'newark','nm':'albuquerque',
           'ny':'new york','nc':'charlotte','nd':'fargo','oh':'columbus','ok':'oklahoma city','or':'portland','pa':'philadelphia',
           'ri':'providence','sc':'charleston','sd':'sioux falls','tn':'nashville','tx':'houston','ut':'salt lake city',
           'vt':'burlington','va':'virginia beach','wa':'seattle','wv':'charleston','wi':'milwaukee','wy':'cheyenne'}

json_data_list = {}
count=0
for key,value in city_list.items():
    if count == 20:
        break
    location = value+","+key
    querystring = {"location":location}

    headers = {
        "X-RapidAPI-Key": "61436d91d8mshc4eb9af52af69f0p115c3ejsn741d81b96d50",
        "X-RapidAPI-Host": "zillow56.p.rapidapi.com"
    }
    try:
        response = requests.get(url, headers=headers, params=querystring)

        json_data = response.json()['data']['marketPage']['medianRentPriceOverTime']
        json_data_list[location]=json_data
    except KeyError:
        print(location)
    count+=1

output_file_path = "json_data_list.json"

try:
    with open(output_file_path, 'r') as file:
        existing_data = json.load(file)
except FileNotFoundError:
    # If the file doesn't exist, initialize with an empty list or dictionary
    existing_data = []

# Append the new data to the existing data
existing_data.append(new_data)

# Write the combined data back to the file
with open(output_file_path, 'w') as file:
    json.dump(existing_data, file)

birmingham,al
anchorage,ak
phoenix,az
little rock,ar
los angeles,ca
denver,co
bridgeport,ct
wilmington,de
jacksonville,fl
atlanta,ga
honolulu,hi
boise,id
chicago,il
indianapolis,in
des moines,ia
wichita,ks
louisville,ky
new orleans,la
portland,me
baltimore,md


AttributeError: 'dict' object has no attribute 'append'

In [7]:
import json

In [5]:
file_path = 'json_data_list.json'
with open(file_path, 'r') as file:
    data = json.load(file)
print(data)

{}


In [18]:
# postman
import requests

url = "https://api.bridgedataoutput.com/api/v2/zgecon/type?access_token=ef3256b5372a3db7e40ff4d6a4f1a801&metadataType=regionType"

payload = ""
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


{"success":false,"status":401,"bundle":{"name":"AuthorizationError","message":"Unauthorized request"}}


In [ ]:
rent_index_path = 'https://files.zillowstatic.com/research/public_csvs/zori/Metro_zori_uc_sfrcondomfr_sm_month.csv?t=1708817451'


In [ ]:
# https://www.forbes.com/advisor/mortgages/average-rent-by-state/
# average rage rent by state
# by scraping data-row_id="155280"